In [1]:
import tensorflow as tf

In [6]:
from tensorflow.python.layers.core import Dense

In [46]:
tf.reset_default_graph()

In [49]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    # Helper for the training process. Used by BasicDecoder to read inputs.


        # Helper for the training process. Used by BasicDecoder to read inputs.
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                            sequence_length=target_sequence_length,
                                                            time_major=False)
        
    drop_cell =  tf.contrib.rnn.DropoutWrapper(dec_cell, output_keep_prob= keep_prob)     
        # Basic decoder
    training_decoder = tf.contrib.seq2seq.BasicDecoder(drop_cell,
                                                           training_helper,
                                                           encoder_state,
                                                           output_layer) 
    
    #print([x.name for x in tf.global_variables()])    
    
        # Perform dynamic decoding using the decoder
    training_decoder_output = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                       impute_finished=True, scope= 'what',
                                                                       maximum_iterations=max_summary_length)[0]
    print([x.name for x in tf.global_variables()]) 
    
    return training_decoder_output

In [50]:
batch_size = 64
vocab_size = 1000
embedding_size = 200
sequence_length = 22
rnn_size = 512
num_layers = 3

with tf.Graph().as_default():
    with tf.variable_scope("decoding") as decoding_scope:
            # dec_cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.BasicLSTMCell(rnn_size)] * num_layers)

        dec_embed_input = tf.placeholder(tf.float32, [batch_size, sequence_length, embedding_size])
        keep_prob = tf.placeholder(tf.float32)
        target_sequence_length_p = tf.placeholder(tf.int32, (None,), name='target_sequence_length')
        max_target_sequence_length = tf.reduce_max(target_sequence_length_p, name='max_target_len')

        for layer in range(num_layers):
            with tf.variable_scope('decoder_{}'.format(layer)):
                lstm = tf.contrib.rnn.LSTMCell(rnn_size,
                                                   initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
                dec_cell = tf.contrib.rnn.DropoutWrapper(lstm,
                                                             input_keep_prob=keep_prob)

        output_layer = Dense(vocab_size,
                                 kernel_initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1),
                                 name='output_layer')
            # output_fn = lambda x: tf.contrib.layers.fully_connected(x, vocab_size, None, scope=decoding_scope)

        encoder_state = tf.contrib.rnn.LSTMStateTuple(
                tf.placeholder(tf.float32, [None, rnn_size]),
                tf.placeholder(tf.float32, [None, rnn_size]))

        train_decoder_output = decoding_layer_train(encoder_state, dec_cell,
                                        dec_embed_input,
                                        target_sequence_length_p,
                                        max_target_sequence_length,
                                        output_layer,
                                        keep_prob)

['decoding/what/lstm_cell/weights:0', 'decoding/what/lstm_cell/biases:0', 'decoding/output_layer/kernel:0', 'decoding/output_layer/bias:0']


In [19]:
train_decoder_output.rnn_output.get_shape().as_list()

[64, None, 1000]

In [20]:
train_decoder_output.rnn_output.get_shape()

TensorShape([Dimension(64), Dimension(None), Dimension(1000)])

In [21]:
train_decoder_output.count

<function BasicDecoderOutput.count>

In [22]:
print([x.name for x in tf.global_variables()]) 

[]


In [30]:
tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'decoding')

[]

In [39]:
tf.reset_default_graph()

In [40]:
target_vocab_size = 10

In [41]:
def make_cell(rnn_size):
        dec_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        
        return dec_cell

dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
output_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))

In [42]:
tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

[]

In [43]:
print([x.name for x in tf.global_variables()]) 

[]
